# Gemini Fullstack LangGraph Quickstart - Agent Architecture Demo

This notebook demonstrates the core agent architecture used in the backend. It visualizes the graph and runs a simple query through the agent.

In [ ]:
# Setup environment
import os
import sys
from dotenv import load_dotenv

# Add backend/src to path
sys.path.append(os.path.abspath("../backend/src"))

load_dotenv("../backend/.env")

# Verify API keys are present (optional check)
if not os.getenv("GOOGLE_API_KEY"):
    print("Warning: GOOGLE_API_KEY not found in environment")

## 1. Visualize the Agent Graph

We'll load the main agent graph and visualize its structure using Mermaid.

In [ ]:
from agent.graph import graph
from IPython.display import Image, display

try:
    # Generate Mermaid PNG
    png_data = graph.get_graph().draw_mermaid_png()
    display(Image(png_data))
except Exception as e:
    print(f"Could not draw graph: {e}")
    # Fallback to ascii
    graph.get_graph().print_ascii()

## 2. Run a Simple Query

Let's run a basic query through the agent to see the step-by-step execution.

In [ ]:
import asyncio
from langchain_core.messages import HumanMessage

async def run_demo():
    input_message = HumanMessage(content="What is the capital of France?")
    config = {"configurable": {"thread_id": "demo_notebook_1"}}
    
    print(f"User: {input_message.content}\n")
    
    async for event in graph.astream_events(
        {"messages": [input_message]},
        config,
        version="v2"
    ):
        kind = event["event"]
        if kind == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                print(content, end="", flush=True)
        elif kind == "on_tool_start":
            print(f"\n[Tool Start: {event['name']}]", flush=True)
        elif kind == "on_tool_end":
            print(f"\n[Tool End: {event['name']}]", flush=True)

# Run the async function
# Note: In Jupyter, you can usually await directly. If running as script, use asyncio.run()
await run_demo()